<a href="https://colab.research.google.com/github/Tinku4-ux/CodeAlpha-/blob/main/inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1 - imports
import os
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

print("TF version:", tf.__version__)


TF version: 2.19.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving test.zip to test.zip
Saving train.csv to train.csv
Saving train.zip to train.zip


In [8]:
# Cell 2 - paths (edit here if needed)
TRAIN_CSV = '/content/train.csv'
TEST_CSV  = '/content/test.csv'
IMG_ROOT  = '/content/'   # base path that contains 'train/' and 'test/' folders
TRAIN_IMG_FOLDER = os.path.join(IMG_ROOT, 'train')
TEST_IMG_FOLDER  = os.path.join(IMG_ROOT, 'test')

print("Train csv exists:", os.path.exists(TRAIN_CSV))
print("Test csv exists:",  os.path.exists(TEST_CSV))
print("Train folder exists:", os.path.isdir(TRAIN_IMG_FOLDER))
print("Test folder exists:", os.path.isdir(TEST_IMG_FOLDER))

# list few images (if many, it'll show the first few)
if os.path.isdir(TRAIN_IMG_FOLDER):
    print("Example train images:", os.listdir(TRAIN_IMG_FOLDER)[:6])


Train csv exists: True
Test csv exists: True
Train folder exists: False
Test folder exists: False


In [9]:
!unzip train.zip -d /content/
!unzip test.zip -d /content/


Archive:  train.zip
   creating: /content/train/
  inflating: /content/train/ID1011485656.jpg  
  inflating: /content/train/ID1012260530.jpg  
  inflating: /content/train/ID1025234388.jpg  
  inflating: /content/train/ID1028611175.jpg  
  inflating: /content/train/ID1035947949.jpg  
  inflating: /content/train/ID1036339023.jpg  
  inflating: /content/train/ID1049634115.jpg  
  inflating: /content/train/ID1051144034.jpg  
  inflating: /content/train/ID1052620238.jpg  
  inflating: /content/train/ID105271783.jpg  
  inflating: /content/train/ID1053972079.jpg  
  inflating: /content/train/ID1058383417.jpg  
  inflating: /content/train/ID1062837331.jpg  
  inflating: /content/train/ID1070112260.jpg  
  inflating: /content/train/ID1078930021.jpg  
  inflating: /content/train/ID1084819986.jpg  
  inflating: /content/train/ID1088965591.jpg  
  inflating: /content/train/ID1098771283.jpg  
  inflating: /content/train/ID1103883611.jpg  
  inflating: /content/train/ID1108283583.jpg  
  inflating:

In [10]:
!ls /content


drive  sample_data  test  test.csv  test.zip  train  train.csv	train.zip


In [11]:
# Cell 2 - paths (edit here if needed)
TRAIN_CSV = '/content/train.csv'
TEST_CSV  = '/content/test.csv'
IMG_ROOT  = '/content/'   # base path that contains 'train/' and 'test/' folders
TRAIN_IMG_FOLDER = os.path.join(IMG_ROOT, 'train')
TEST_IMG_FOLDER  = os.path.join(IMG_ROOT, 'test')

print("Train csv exists:", os.path.exists(TRAIN_CSV))
print("Test csv exists:",  os.path.exists(TEST_CSV))
print("Train folder exists:", os.path.isdir(TRAIN_IMG_FOLDER))
print("Test folder exists:", os.path.isdir(TEST_IMG_FOLDER))

if os.path.isdir(TRAIN_IMG_FOLDER):
    print("Example train images:", os.listdir(TRAIN_IMG_FOLDER)[:6])


Train csv exists: True
Test csv exists: True
Train folder exists: True
Test folder exists: True
Example train images: ['ID50027657.jpg', 'ID1345375788.jpg', 'ID668475812.jpg', 'ID1159071020.jpg', 'ID157479394.jpg', 'ID2056023629.jpg']


In [12]:
# Cell 3 - load CSV and inspect
train_df = pd.read_csv('/content/train.csv')
test_df  = pd.read_csv('/content/test.csv')

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

print("\nTrain Columns:\n", train_df.columns.tolist())
display(train_df.head(10))


Train shape: (1785, 9)
Test shape: (5, 3)

Train Columns:
 ['sample_id', 'image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm', 'target_name', 'target']


,sample_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Clover_g,0.0000
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Dead_g,31.9984
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Green_g,16.2751
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Total_g,48.2735
4,ID1011485656__GDM_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,GDM_g,16.2750
5,ID1012260530__Dry_Clover_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,Dry_Clover_g,0.0000
6,ID1012260530__Dry_Dead_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,Dry_Dead_g,0.0000
7,ID1012260530__Dry_Green_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,Dry_Green_g,7.6000
8,ID1012260530__Dry_Total_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,Dry_Total_g,7.6000
9,ID1012260530__GDM_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,GDM_g,7.6000


In [13]:
# Cell 4 - Convert LONG format (5 rows per image) → WIDE format (1 row per image)

# Extract image_id from sample_id (everything before __ )
train_df["image_id"] = train_df["sample_id"].apply(lambda x: x.split("__")[0])

# Pivot the dataset: rows = image_id, columns = target_name
pivot_targets = train_df.pivot_table(
    index=["image_id", "image_path", "Sampling_Date", "State", "Species", "Pre_GSHH_NDVI", "Height_Ave_cm"],
    columns="target_name",
    values="target"
).reset_index()

# Fix column names (target_name becomes proper column)
pivot_targets.columns.name = None

print("Pivoted Shape:", pivot_targets.shape)
pivot_targets.head(10)


Pivoted Shape: (357, 12)


,image_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g
0,ID1011485656,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,0.0000,31.9984,16.2751,48.2735,16.2750
1,ID1012260530,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,0.0000,0.0000,7.6000,7.6000,7.6000
2,ID1025234388,train/ID1025234388.jpg,2015/9/1,WA,SubcloverDalkeith,0.38,1.0000,6.0500,0.0000,0.0000,6.0500,6.0500
3,ID1028611175,train/ID1028611175.jpg,2015/5/18,Tas,Ryegrass,0.66,5.0000,0.0000,30.9703,24.2376,55.2079,24.2376
4,ID1035947949,train/ID1035947949.jpg,2015/9/11,Tas,Ryegrass,0.54,3.5000,0.4343,23.2239,10.5261,34.1844,10.9605
5,ID1036339023,train/ID1036339023.jpg,2015/9/30,Vic,Phalaris_Clover,0.82,7.0000,23.0755,2.6135,32.1910,57.8800,55.2665
6,ID1049634115,train/ID1049634115.jpg,2015/7/2,Vic,Ryegrass_Clover,0.72,2.0000,1.5083,3.0167,13.5750,18.1000,15.0833
7,ID1051144034,train/ID1051144034.jpg,2015/9/1,WA,SubcloverLosa,0.80,1.0000,55.3200,0.0000,0.0000,55.3200,55.3200
8,ID1052620238,train/ID1052620238.jpg,2015/5/18,Tas,Ryegrass,0.68,5.0000,0.0000,11.2291,20.1707,31.3998,20.1707
9,ID105271783,train/ID105271783.jpg,2015/6/30,Vic,Phalaris_Clover,0.86,4.0000,5.2698,8.5635,27.6667,41.5000,32.9365


In [14]:
# Cell 5 - Prepare final dataset

df = pivot_targets.copy()

# 1. Create full image path
df['img_fullpath'] = df['image_path'].apply(lambda x: os.path.join('/content', x))

# 2. Extract year and month from Sampling_Date
df['Sampling_Date'] = pd.to_datetime(df['Sampling_Date'], errors='coerce')
df['year'] = df['Sampling_Date'].dt.year
df['month'] = df['Sampling_Date'].dt.month

# 3. Encode categorical metadata (State, Species)
df['State'] = df['State'].astype('category').cat.codes
df['Species'] = df['Species'].astype('category').cat.codes

# 4. Define target columns
TARGET_COLS = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']

# 5. Log1p transform for RMSLE
for c in TARGET_COLS:
    df['log_' + c] = np.log1p(df[c].astype(np.float32))

LOG_TARGET_COLS = ['log_' + c for c in TARGET_COLS]

# 6. Define metadata columns (NDVI, Height, State, Species, year, month)
META_COLS = ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'State', 'Species', 'year', 'month']

print("Final DataFrame shape:", df.shape)
print("\nFinal columns:", df.columns.tolist())
df.head(10)


Final DataFrame shape: (357, 20)

Final columns: ['image_id', 'image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm', 'Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g', 'img_fullpath', 'year', 'month', 'log_Dry_Clover_g', 'log_Dry_Dead_g', 'log_Dry_Green_g', 'log_Dry_Total_g', 'log_GDM_g']


,image_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g,img_fullpath,year,month,log_Dry_Clover_g,log_Dry_Dead_g,log_Dry_Green_g,log_Dry_Total_g,log_GDM_g
0,ID1011485656,train/ID1011485656.jpg,2015-09-04,1,11,0.62,4.6667,0.0000,31.9984,16.2751,48.2735,16.2750,/content/train/ID1011485656.jpg,2015,9,0.000000,3.496459,2.849266,3.897386,2.849260
1,ID1012260530,train/ID1012260530.jpg,2015-04-01,0,3,0.55,16.0000,0.0000,0.0000,7.6000,7.6000,7.6000,/content/train/ID1012260530.jpg,2015,4,0.000000,0.000000,2.151762,2.151762,2.151762
2,ID1025234388,train/ID1025234388.jpg,2015-09-01,3,12,0.38,1.0000,6.0500,0.0000,0.0000,6.0500,6.0500,/content/train/ID1025234388.jpg,2015,9,1.953028,0.000000,0.000000,1.953028,1.953028
3,ID1028611175,train/ID1028611175.jpg,2015-05-18,1,10,0.66,5.0000,0.0000,30.9703,24.2376,55.2079,24.2376,/content/train/ID1028611175.jpg,2015,5,0.000000,3.464807,3.228335,4.029058,3.228335
4,ID1035947949,train/ID1035947949.jpg,2015-09-11,1,10,0.54,3.5000,0.4343,23.2239,10.5261,34.1844,10.9605,/content/train/ID1035947949.jpg,2015,9,0.360677,3.187340,2.444614,3.560603,2.481610
5,ID1036339023,train/ID1036339023.jpg,2015-09-30,2,7,0.82,7.0000,23.0755,2.6135,32.1910,57.8800,55.2665,/content/train/ID1036339023.jpg,2015,9,3.181195,1.284677,3.502279,4.075501,4.030099
6,ID1049634115,train/ID1049634115.jpg,2015-07-02,2,11,0.72,2.0000,1.5083,3.0167,13.5750,18.1000,15.0833,/content/train/ID1049634115.jpg,2015,7,0.919605,1.390461,2.679308,2.949688,2.777781
7,ID1051144034,train/ID1051144034.jpg,2015-09-01,3,13,0.80,1.0000,55.3200,0.0000,0.0000,55.3200,55.3200,/content/train/ID1051144034.jpg,2015,9,4.031050,0.000000,0.000000,4.031050,4.031050
8,ID1052620238,train/ID1052620238.jpg,2015-05-18,1,10,0.68,5.0000,0.0000,11.2291,20.1707,31.3998,20.1707,/content/train/ID1052620238.jpg,2015,5,0.000000,2.503818,3.052618,3.478152,3.052618
9,ID105271783,train/ID105271783.jpg,2015-06-30,2,7,0.86,4.0000,5.2698,8.5635,27.6667,41.5000,32.9365,/content/train/ID105271783.jpg,2015,6,1.835745,2.257954,3.355736,3.749504,3.524491


In [15]:
# Cell 6 - Create GroupKFold splits using image_id (best grouping)

from sklearn.model_selection import GroupKFold

N_FOLDS = 5
df['fold'] = -1

groups = df['image_id'].values

gkf = GroupKFold(n_splits=N_FOLDS)
for fold, (train_idx, val_idx) in enumerate(gkf.split(df, df[TARGET_COLS], groups)):
    df.loc[val_idx, 'fold'] = fold

print(df['fold'].value_counts().sort_index())
df[['image_id', 'fold']].head(10)


fold
0    72
1    72
2    71
3    71
4    71
Name: count, dtype: int64


,image_id,fold
0,ID1011485656,0
1,ID1012260530,1
2,ID1025234388,2
3,ID1028611175,3
4,ID1035947949,4
5,ID1036339023,0
6,ID1049634115,1
7,ID1051144034,2
8,ID1052620238,3
9,ID105271783,4


In [16]:
# Cell 7 - TF Dataset Pipeline

IMG_SIZE = 224
BATCH_SIZE = 16

# Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.05),
    tf.keras.layers.RandomContrast(0.1),
])

# Function to load & preprocess image
def load_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img

# Full dataset builder
def build_dataset(dataframe, training=True):
    img_paths = dataframe['img_fullpath'].values
    metadata = dataframe[META_COLS].values.astype('float32')
    labels = dataframe[LOG_TARGET_COLS].values.astype('float32')

    ds = tf.data.Dataset.from_tensor_slices((img_paths, metadata, labels))

    def process(path, meta, label):
        img = load_image(path)
        if training:
            img = data_augmentation(img)
        return (img, meta), label

    if training:
        ds = ds.shuffle(2048)

    ds = ds.map(process, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

# Test the pipeline with fold=0
train_df_fold0 = df[df['fold'] != 0].reset_index(drop=True)
val_df_fold0   = df[df['fold'] == 0].reset_index(drop=True)

train_ds = build_dataset(train_df_fold0, training=True)
val_ds   = build_dataset(val_df_fold0, training=False)

for (imgs, metas), labels in train_ds.take(1):
    print("Images:", imgs.shape)
    print("Metadata:", metas.shape)
    print("Labels:", labels.shape)


Images: (16, 224, 224, 3)
Metadata: (16, 6)
Labels: (16, 5)


In [17]:
# Cell 8 - Build Multi-Input Multi-Output Model (EfficientNetB3 + Metadata)

def build_model(img_size=IMG_SIZE, n_meta=len(META_COLS), n_targets=len(TARGET_COLS)):

    # IMAGE BRANCH
    base_model = tf.keras.applications.EfficientNetB3(
        include_top=False,
        weights="imagenet",
        input_shape=(img_size, img_size, 3)
    )
    base_model.trainable = False  # Stage 1: freeze

    img_input = tf.keras.Input(shape=(img_size, img_size, 3), name='image_input')
    x = base_model(img_input, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)

    # METADATA BRANCH
    meta_input = tf.keras.Input(shape=(n_meta,), name='meta_input')
    m = tf.keras.layers.BatchNormalization()(meta_input)
    m = tf.keras.layers.Dense(64, activation='relu')(m)
    m = tf.keras.layers.Dense(32, activation='relu')(m)

    # COMBINE BOTH BRANCHES
    combined = tf.keras.layers.Concatenate()([x, m])
    combined = tf.keras.layers.Dense(128, activation='relu')(combined)
    combined = tf.keras.layers.Dropout(0.2)(combined)

    # FINAL MULTI-OUTPUT LAYER (predicts log1p values)
    output = tf.keras.layers.Dense(n_targets, name='output')(combined)

    model = tf.keras.Model(inputs=[img_input, meta_input], outputs=output)
    return model


# Build the model
model = build_model()
model.summary()


43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb3      │ (None, 7, 7,      │ 10,783,535 │ image_input[0][0] │
│ (Functional)        │ 1536)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ meta_input          │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1536)      │          0 │ efficientnetb3[0… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 6)         │         24 │ meta_input[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1536)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │        448 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    393,472 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      2,080 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 288)       │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     36,992 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 5)         │        645 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,217,196 (42.79 MB)

 Trainable params: 433,649 (1.65 MB)

 Non-trainable params: 10,783,547 (41.14 MB)

In [18]:
# Cell 9 - Compile model

# RMSLE metric on log-space
def rmsle_metric(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=[rmsle_metric, 'mae']
)

print("Model compiled successfully.")


Model compiled successfully.


In [20]:
# Cell 10 - Stage 1 Training (Head Only) — FIXED

FOLD = 0

train_df_fold = df[df['fold'] != FOLD].reset_index(drop=True)
val_df_fold   = df[df['fold'] == FOLD].reset_index(drop=True)

train_ds = build_dataset(train_df_fold, training=True)
val_ds   = build_dataset(val_df_fold, training=False)

# Rebuild model (ensure freeze)
model = build_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='mse',
    metrics=[rmsle_metric]
)

checkpoint_path = "/content/best_stage1_fold0.weights.h5"

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        patience=3,
        factor=0.5,
        min_lr=1e-6
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=6,
        restore_best_weights=True
    )
]

history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=12,
    callbacks=callbacks_list
)


Epoch 1/12
18/18 ━━━━━━━━━━━━━━━━━━━━ 95s 4s/step - loss: 3.5650 - rmsle_metric: 1.7701 - val_loss: 3573.3901 - val_rmsle_metric: 59.7493 - learning_rate: 0.0010
Epoch 2/12
18/18 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - loss: 1.2215 - rmsle_metric: 1.0978 - val_loss: 3462.5068 - val_rmsle_metric: 58.8102 - learning_rate: 0.0010
Epoch 3/12
18/18 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - loss: 1.1534 - rmsle_metric: 1.0688 - val_loss: 3821.4575 - val_rmsle_metric: 61.7826 - learning_rate: 0.0010
Epoch 4/12
18/18 ━━━━━━━━━━━━━━━━━━━━ 73s 4s/step - loss: 0.9362 - rmsle_metric: 0.9578 - val_loss: 4235.7847 - val_rmsle_metric: 65.0430 - learning_rate: 0.0010
Epoch 5/12
18/18 ━━━━━━━━━━━━━━━━━━━━ 72s 3s/step - loss: 0.8924 - rmsle_metric: 0.9399 - val_loss: 4223.2031 - val_rmsle_metric: 64.9445 - learning_rate: 0.0010
Epoch 6/12
18/18 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - loss: 0.8249 - rmsle_metric: 0.9020 - val_loss: 3825.0876 - val_rmsle_metric: 61.8033 - learning_rate: 5.0000e-04
Epoch 7/12
18/18 ━━━━━━━